In [1]:
!pip install pyspark

  Using cached pyspark-3.4.1.tar.gz (310.8 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285432 sha256=f267b2588dac9b405e532a40129e094ad6a89bb5c0da0069a04953eca1aab761
  Stored in directory: c:\users\rose des vents\appdata\local\pip\cache\wheels\0d\77\a3\ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [5]:
import os
import pandas as pd
import numpy as np

In [3]:
#!pip install numpy

  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/b7/db/4d37359e2c9cf8bf071c08b8a6f7374648a5ab2e76e2e22e3b808f81d507/numpy-1.25.2-cp310-cp310-win_amd64.whl.metadata
   ---------------------------------------- 0.0/15.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/15.6 MB 656.4 kB/s eta 0:00:24
   ---------------------------------------- 0.2/15.6 MB 1.4 MB/s eta 0:00:11
    --------------------------------------- 0.3/15.6 MB 1.7 MB/s eta 0:00:09
   - -------------------------------------- 0.6/15.6 MB 2.7 MB/s eta 0:00:06
   -- ------------------------------------- 0.9/15.6 MB 3.3 MB/s eta 0:00:05
   -- ------------------------------------- 1.1/15.6 MB 3.5 MB/s eta 0:00:05
   --- ------------------------------------ 1.3/15.6 MB 3.6 MB/s eta 0:00:04
   --- ------------------------------------ 

In [4]:
#!pip install pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/c5/89/ce1c7dc497f9a20644f6a7d2dd5bce6378a48321955178197fa3b55d6fe3/pandas-2.1.0-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/32/4d/aaf7eff5deb402fd9a24a1449a8119f00d74ae9c2efa79f8ef9994261fc2/pytz-2023.3.post1-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/341.8 kB ? eta -:--:--
     -------- ------------------------------ 71.7/341.8 kB 2.0 MB/s eta 0:00:01
     ----------------- -------------------- 153.6/341.8 kB 1.8 MB/s eta 0:00:01
     ----------------------------------- -- 317.4/341.8 kB 2.5 MB/s eta 0:00:01
     -------------------------------------- 341.8/341.8 kB 2.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.1 MB 2.6 MB/s eta 0:00:05
   - -------------------------------------

In [6]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

In [7]:
from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator

In [10]:
import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
#!pip install seaborn

     ---------------------------------------- 0.0/293.3 kB ? eta -:--:--
     - -------------------------------------- 10.2/293.3 kB ? eta -:--:--
     - -------------------------------------- 10.2/293.3 kB ? eta -:--:--
     --- --------------------------------- 30.7/293.3 kB 262.6 kB/s eta 0:00:01
     ------------- ---------------------- 112.6/293.3 kB 656.4 kB/s eta 0:00:01
     ------------------------- ---------- 204.8/293.3 kB 958.4 kB/s eta 0:00:01
     -------------------------------------- 293.3/293.3 kB 1.2 MB/s eta 0:00:00
  Obtaining dependency information for matplotlib!=3.6.1,>=3.1 from https://files.pythonhosted.org/packages/7a/94/a1615bac6706eb8a58fe08675b80a11b5463ef9591176dd3e57257495774/matplotlib-3.8.0-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/94/0a/5eb57dd395fade977786b2d2c98c2bee8234358794be44422fe58a719d42/contourpy-1.1.0-cp310-cp310-win_amd64.whl.metadata
  Obtaining d

In [11]:
# Visualization
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 400)

from matplotlib import rcParams
sns.set(context='notebook', style='whitegrid', rc={'figure.figsize': (18,4)})
rcParams['figure.figsize'] = 18,4

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [12]:
#creating session spark
spark = SparkSession.builder.master("local[2]").appName("classifier.mushroom").getOrCreate()

In [13]:
spark

In [14]:
sc = spark.sparkContext
sc

<SparkContext master=local[2] appName=classifier.mushroom>

In [15]:
sqlContext = SQLContext(spark.sparkContext)
sqlContext

c:\Users\Rose des vents\anaconda3\envs\Spark\lib\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [19]:
df_pyspark = spark.read.csv("mushrooms.csv",inferSchema=True, header=True)

In [20]:
df_pyspark.show(5)

+-----+---------+-----------+---------+-------+----+---------------+------------+---------+----------+-----------+----------+------------------------+------------------------+----------------------+----------------------+---------+----------+-----------+---------+-----------------+----------+-------+
|class|cap-shape|cap-surface|cap-color|bruises|odor|gill-attachment|gill-spacing|gill-size|gill-color|stalk-shape|stalk-root|stalk-surface-above-ring|stalk-surface-below-ring|stalk-color-above-ring|stalk-color-below-ring|veil-type|veil-color|ring-number|ring-type|spore-print-color|population|habitat|
+-----+---------+-----------+---------+-------+----+---------------+------------+---------+----------+-----------+----------+------------------------+------------------------+----------------------+----------------------+---------+----------+-----------+---------+-----------------+----------+-------+
|    p|        x|          s|        n|      t|   p|              f|           c|        n|   

In [25]:
df_pyspark.printSchema()

root
 |-- class: string (nullable = true)
 |-- cap-shape: string (nullable = true)
 |-- cap-surface: string (nullable = true)
 |-- cap-color: string (nullable = true)
 |-- bruises: string (nullable = true)
 |-- odor: string (nullable = true)
 |-- gill-attachment: string (nullable = true)
 |-- gill-spacing: string (nullable = true)
 |-- gill-size: string (nullable = true)
 |-- gill-color: string (nullable = true)
 |-- stalk-shape: string (nullable = true)
 |-- stalk-root: string (nullable = true)
 |-- stalk-surface-above-ring: string (nullable = true)
 |-- stalk-surface-below-ring: string (nullable = true)
 |-- stalk-color-above-ring: string (nullable = true)
 |-- stalk-color-below-ring: string (nullable = true)
 |-- veil-type: string (nullable = true)
 |-- veil-color: string (nullable = true)
 |-- ring-number: string (nullable = true)
 |-- ring-type: string (nullable = true)
 |-- spore-print-color: string (nullable = true)
 |-- population: string (nullable = true)
 |-- habitat: string 

In [22]:
from pyspark.ml.feature import StringIndexer

In [23]:
columns = df_pyspark.columns

In [27]:
for categoricalCol in columns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol+"_encoded").fit(df_pyspark)
    df_pyspark = stringIndexer.transform(df_pyspark)
    df_pyspark = df_pyspark.withColumn(categoricalCol+"_encoded", df_pyspark[categoricalCol+"_encoded"].cast('int'))

In [28]:
col_encoded = [col+'_encoded'for col in columns]

encoded_df=df_pyspark.select(col_encoded)


In [29]:
encoded_df.show(5)

+-------------+-----------------+-------------------+-----------------+---------------+------------+-----------------------+--------------------+-----------------+------------------+-------------------+------------------+--------------------------------+--------------------------------+------------------------------+------------------------------+-----------------+------------------+-------------------+-----------------+-------------------------+------------------+---------------+
|class_encoded|cap-shape_encoded|cap-surface_encoded|cap-color_encoded|bruises_encoded|odor_encoded|gill-attachment_encoded|gill-spacing_encoded|gill-size_encoded|gill-color_encoded|stalk-shape_encoded|stalk-root_encoded|stalk-surface-above-ring_encoded|stalk-surface-below-ring_encoded|stalk-color-above-ring_encoded|stalk-color-below-ring_encoded|veil-type_encoded|veil-color_encoded|ring-number_encoded|ring-type_encoded|spore-print-color_encoded|population_encoded|habitat_encoded|
+-------------+-------------

In [30]:
from pyspark.ml.feature import VectorAssembler

In [31]:
col_encoded.remove('class_encoded')

In [33]:
featureAssembler = VectorAssembler(inputCols=col_encoded,outputCol="features")

In [34]:
output = featureAssembler.transform(encoded_df)

In [35]:
output.select("features","class_encoded").show(5)

+--------------------+-------------+
|            features|class_encoded|
+--------------------+-------------+
|(22,[1,3,4,7,8,9,...|            1|
|(22,[1,2,3,4,8,9,...|            0|
|(22,[0,1,2,3,4,8,...|            0|
|(22,[2,3,4,7,8,9,...|            1|
|(22,[1,2,6,8,10,1...|            0|
+--------------------+-------------+
only showing top 5 rows



In [40]:
nombre_de_lignes = df_pyspark.count()
nombre_de_lignes

8124

In [44]:
train, test = output.randomSplit([0.8, 0.2], seed=17)

In [45]:
print("size of training data:", train.count())
print("Size of testing data:",test.count())

size of training data: 6471
Size of testing data: 1653


In [46]:
from pyspark.ml.classification import LogisticRegression

In [49]:
lr = LogisticRegression(featuresCol = 'features', labelCol = 'class_encoded', maxIter=10)

In [50]:
lrModel = lr.fit(train)

In [51]:
predictions = lrModel.transform(test)

In [52]:
predictions.show(5)

+-------------+-----------------+-------------------+-----------------+---------------+------------+-----------------------+--------------------+-----------------+------------------+-------------------+------------------+--------------------------------+--------------------------------+------------------------------+------------------------------+-----------------+------------------+-------------------+-----------------+-------------------------+------------------+---------------+--------------------+--------------------+--------------------+----------+
|class_encoded|cap-shape_encoded|cap-surface_encoded|cap-color_encoded|bruises_encoded|odor_encoded|gill-attachment_encoded|gill-spacing_encoded|gill-size_encoded|gill-color_encoded|stalk-shape_encoded|stalk-root_encoded|stalk-surface-above-ring_encoded|stalk-surface-below-ring_encoded|stalk-color-above-ring_encoded|stalk-color-below-ring_encoded|veil-type_encoded|veil-color_encoded|ring-number_encoded|ring-type_encoded|spore-print-colo

In [53]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [54]:
evaluator = MulticlassClassificationEvaluator()

In [55]:
evaluator.setLabelCol("class_encoded")

MulticlassClassificationEvaluator_b07e77837852

In [56]:
evaluator.setPredictionCol("prediction")

MulticlassClassificationEvaluator_b07e77837852

In [57]:
evaluator.evaluate(predictions)

0.9915305505142167

In [61]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator=MulticlassClassificationEvaluator()
evaluator.setLabelCol("class_encoded")
evaluator.setPredictionCol("prediction")
print(" test Area Under ROC:", evaluator.evaluate(predictions))

MulticlassClassificationEvaluator_f5cac889e758

MulticlassClassificationEvaluator_f5cac889e758

 test Area Under ROC: 0.9915305505142167
